In [9]:
#CONVERT PDF TO PLAINTEXT FILES
import pdfminer
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from six import StringIO
import re
import sys
import os
sys.path.append("/usr/local/lib/python3.7/site-packages/pdfminer")

def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()
    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)
    fp.close()
    device.close()
    str = retstr.getvalue()
    retstr.close()
    return str.replace("\\n","\n")

#Open each PDF file in the folder and convert into plaintext
Path = "./Data/Fulltext_text_PDFs/"
Output_Path = "./Data/Fulltext_Plaintext/"

#read the PDF files to get plain text full-text
filelist = os.listdir(Path)
for i in filelist:
    if i.endswith(".pdf"):  
        with open(Path + i, 'r') as f:
            file_path = Path + i
            new_file_name = i.replace(".pdf",".txt")
            text_file_name = Output_Path + new_file_name    
            text = convert_pdf_to_txt(file_path)
            article_plaintext = open(text_file_name,'w')  
            article_plaintext.write(text) 
            article_plaintext.close()
            
            

ModuleNotFoundError: No module named 'pdfminer'

In [6]:
from io import BytesIO as StringIO
import re
import sys
import os

#GET THE METHODS SECTIONS TO PLAINTEXT FILES
Output_Path = "./Data/Fulltext_Plaintext/"
#Get the text between Methods & Results section 
filelist_2 = os.listdir(Output_Path)
start_pattern = ["METHODS","Methods","Subjects and methods","Subjects and Methods","MATERIALS AND METHODS","Materials and methods","METHODS AND MATERIALS","PATIENTS  AND  METHODS",
                 "PARTICIPANTS AND METHODS","SUBJECTS AND METHODS","Materials and Methods"]
stop_pattern = ["RESULTS","Results","RESULTS AND DISCUSSION"]
for i in filelist_2:
    if i.endswith(".txt"):
        with open(Output_Path + i, 'r') as article_plaintext_file:
            file_path = Output_Path + i
            text_between_Methods_Results = []
            recording = False
            output_section = []
            
            for line in article_plaintext_file:
                line = line.replace("\n", "")
                if recording is False:
                    #If the line contains "METHODS" header, start record the text, change the flag to "TRUE"
                    if line in start_pattern:
                        recording = True
                        output_section.append(line.strip())
                elif recording is True:
                    #If the line contains "RESULTS" header, stop record the text, change the flag to "FALSE"
                    if line in stop_pattern:
                        recording = False
                    else:
                        output_section.append(line.strip())    
                        
            #Save the methods section into a text file
            methods_section_path = './Data/Fulltext_Plaintext/Plaintext_Methods_Sections/'
            methods_section_file_name = methods_section_path + "methods_section" + i
            methods_section_file = open(methods_section_file_name,'w')  
            for line in output_section:
                line = line.strip()
                line = line.replace("\n", " ")
                line = line + " "
                methods_section_file.write(line)           
            methods_section_file.close()
    

In [8]:
#PREPROCESS TEXT OF THE METHODS SECTION

#remove stopword in "Methods section" text files:         
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
#word_tokenize accepts a string as an input, not a file. 
stop_words = set(stopwords.words('english')) 

Methods_Section_Path = "/Users/phuong/Documents/AAAI_Project/Full_Text_Articles/RCT_Full_Text/Methods_Section_Manual_2/"
filelist_3 = os.listdir(Methods_Section_Path)
for i in filelist_3:
    if i.endswith(".txt"):
        with open(Methods_Section_Path + i, 'r') as file1:
            #Preprocess the text from Methods section: eliminate stop words, eliminate numbers.
            line1 = file1.read()# Use this to read file content as a stream
            line1 = line1.replace("- ","")
            line1 = re.sub(' +', ' ',line1)
            words = line1.split() 
            methods_section_file_name_filtered = Methods_Section_Path + "methods_section_filtered_" + i
            for word in words:
                word = re.sub(r'\W+', ' ', word)
                word = word.lstrip()
                word = word.rstrip()
                if not word in stop_words: 
                    if not re.match("^[0-9 ]+$", word):
                        appendFile = open(methods_section_file_name_filtered,'a') 
                        appendFile.write(" "+ word) 
                        appendFile.close()
